In [1]:
import gc
import json
from glob import glob
from pathlib import Path
from typing import Any

import numpy as np
import polars as pl
import torch
from datasets import Dataset
from omegaconf import OmegaConf
from peft import AutoPeftModelForCausalLM, LoraConfig, prepare_model_for_kbit_training
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer


/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_seed(64)

#use bf16 and FlashAttention if supported
compute_dtype = torch.bfloat16
attn_implementation = "flash_attention_2"

## Configuration

In [3]:
config = OmegaConf.load("config.yaml")

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-72B-Instruct"

SEQ_LENGTH = config.max_seq_length
TRAINING_SEQ_LENGTH = config.prompt_max_seq_length
DATA_ROOT = Path(config.data_dir)

OUT_DIR_NAME = Path("./qwne25_72B")

In [5]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, quantization_config=bnb_config, device_map="auto", attn_implementation=attn_implementation, torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.22it/s]


In [6]:
model = prepare_model_for_kbit_training(model, gradient_checkpointing_kwargs={"use_reentrant": True})
model.config.use_cache = False

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["o_proj", "k_proj", "q_proj", "down_proj", "gate_proj", "up_proj", "v_proj"],
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    add_eos_token=True,
    add_bos_token=True,
    )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

## Data Prep

In [8]:
# Load the dataset prepared in `prepare_dataset.ipynb`
train_long = pl.read_parquet("./df_train.parquet")
synth_long = pl.read_parquet("./df_synth.parquet")

In [9]:
train_long = train_long.with_columns(
    MisconceptionId = train_long["MisconceptionId"].cast(pl.Int64),
)

In [10]:
# Feature Engineering: Information related to the problem's category
train_long = train_long.with_columns(
    pl.col("MisconceptionId_pred").str.split(" ").list.eval(pl.element().cast(pl.Int32())),
    knowledge = pl.concat_str(
        [pl.col("FirstSubjectName"), pl.col("SecondSubjectName"),pl.col("ThirdSubjectName"), pl.col("ConstructName")],
        separator=",",
    ),
)

synth_long = synth_long.with_columns(
    pl.col("MisconceptionId_pred").str.split(" ").list.eval(pl.element().cast(pl.Int32())),
    knowledge = pl.concat_str(
        [pl.col("FirstSubjectName"), pl.col("SecondSubjectName"),pl.col("ThirdSubjectName"), pl.col("ConstructName")],
        separator=",",
    ),
)

In [11]:
df_misconceptions = pl.read_csv(DATA_ROOT / "misconception_mapping.csv")
misconception_map = dict(df_misconceptions.rows())

## Make prompts

In [12]:
report_format = """# OBJECTIVE #
Based on the problem, please select which of the following misconceptions might have led to the student's incorrect answer.

Categories: {knowledge}
Question: {question_tex}
Correct Answer: {correct_answer_text}
Student's Answer: {answer_text}
Step-by-Step Solution: {cot_misunderstanding}

Possible Misconceptions:\n{misconceptions_text}

Please respond only with the letter(s) of the misconception(s) you think are most likely to be the cause."""  # noqa: E501


def make_prompt(row: dict[str, Any]) -> tuple[str, str]:
    # Randomly selects a candidate from the top 5 to top 25
    k = np.random.choice([5, 10, 15, 25])
    target_misconceptions = row["MisconceptionId_pred"]
    target_misconceptions = list(target_misconceptions[:k])

    # If the correct answer is not within the Top N options,
    # randomly replace one of the options with the correct answer.
    if row["MisconceptionId"] not in target_misconceptions:
        idx = np.random.randint(0, k)
        target_misconceptions[idx] = row["MisconceptionId"]

    np.random.shuffle(target_misconceptions)

    # Generate the initials
    letters = [chr(i+65) for i in range(k)]
    misconception_names = [misconception_map[_id] for _id in target_misconceptions]
    ans_pos = target_misconceptions.index(row["MisconceptionId"])
    ans_letter = letters[ans_pos]
    choice_str = "\n".join([f"{a}. {n}" for a, n in zip(letters, misconception_names, strict=False)])

    # Prepare a dictionary for validation
    choices = dict(zip(letters, target_misconceptions, strict=False))

    # Make prompt
    q = report_format.format(
        knowledge=row["knowledge"],
        question_tex=row["QuestionText"],
        construct=row["ConstructName"],
        correct_answer_text=row["CorrectAnswerText"],
        answer_text=row["AnswerText"],
        cot_misunderstanding=row["p000-qwen25-32b-instruct-cot_misunderstanding"],
        misconceptions_text=choice_str,
    )
    context = [
        {"role": "system", "content": "You are a mathematics teacher. Your job is to infer and identify the misconceptions behind the incorrect answers to the questions."},
        {
            "role": "user",
            "content": q,
        },
        {
            "role": "assistant",
            "content": ans_letter,
        },
    ]
    prompt: str = tokenizer.apply_chat_template(context, tokenize=False, add_generation_prompt=False)
    return prompt, json.dumps(choices)


In [13]:
# epoch1
np.random.seed(1234)
results = [make_prompt(row) for row in train_long.rows(named=True)]
train_long = train_long.with_columns(
    whole_prompt = pl.Series([ret[0] for ret in results]),
    choices = pl.Series([ret[1] for ret in results]),
)

# epoch2
np.random.seed(1192)
results = [make_prompt(row) for row in train_long.rows(named=True)]
train_long2 = train_long.clone().with_columns(
    whole_prompt = pl.Series([ret[0] for ret in results]),
    choices = pl.Series([ret[1] for ret in results]),
)

In [14]:
results = [make_prompt(row) for row in synth_long.rows(named=True)]
synth_long = synth_long.with_columns(
    whole_prompt = pl.Series([ret[0] for ret in results]),
    choices = pl.Series([ret[1] for ret in results]),
)

In [15]:
# Store the preprocessed prompt for quantization
train_long.write_parquet("./train_long.parquet")

In [16]:
print(train_long["whole_prompt"][0])

<|im_start|>system
You are a mathematics teacher. Your job is to infer and identify the misconceptions behind the incorrect answers to the questions.<|im_end|>
<|im_start|>user
# OBJECTIVE #
Based on the problem, please select which of the following misconceptions might have led to the student's incorrect answer.

Categories: Number,Basic Arithmetic,BIDMAS,Use the order of operations to carry out calculations involving powers
Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
Correct Answer: \( 3 \times(2+4)-5 \)
Student's Answer: Does not need brackets
Step-by-Step Solution: The students' misunderstanding lies in their lack of recognition of the importance of the order of operations, which is governed by the BIDMAS rule (Brackets, Indices, Division/Multiplication, Addition/Subtraction). BIDMAS helps determine the sequence in which arithmetic operations should be performed to ensure the correct result.

In the expression \( 3 \times 2 + 

In [17]:
df_all = pl.concat([
    train_long, synth_long, train_long2,
], how="diagonal_relaxed")

df_all.shape

(22906, 46)

### Data cleansing

In [18]:
# Remove prompts with excessively long token lengths
token_les = [tokenizer(t)["input_ids"] for t in tqdm(df_all["whole_prompt"])]
token_les = [len(t) for t in token_les]
df_all = df_all.with_columns(token_len=pl.Series(token_les))

100%|██████████| 22906/22906 [00:14<00:00, 1591.19it/s]


In [19]:
df_all = df_all.filter(pl.col("token_len") < TRAINING_SEQ_LENGTH)
df_all["token_len"].describe()

statistic,value
str,f64
"""count""",21884.0
"""null_count""",0.0
"""mean""",699.801636
"""std""",144.799527
"""min""",282.0
"""25%""",588.0
"""50%""",689.0
"""75%""",812.0
"""max""",1023.0


### Train the model

In [20]:
fold = 0

# Qwen-72B model was trained using the full dataset.
train_dataset = Dataset.from_polars(df_all).shuffle(seed=0)

valid_dataset = Dataset.from_polars(train_long.filter(train_long["fold"] == fold))

In [21]:
if "Gemma" in type(tokenizer).__name__:
    sep = "<start_of_turn>model\n"
else:
    sep = "<|im_start|>assistant\n"

In [22]:
# LLMの応答部分のスコアのみ計算
collator = DataCollatorForCompletionOnlyLM(response_template=sep, tokenizer=tokenizer, mlm=False)

output_dir = f"{OUT_DIR_NAME}_fold{fold}"

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field = "whole_prompt",
    max_seq_length = SEQ_LENGTH,
    data_collator=collator,
    dataset_num_proc = 8,
    peft_config=peft_config,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 3e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 20,
        save_steps=50,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 334,
        output_dir = output_dir,
        save_total_limit=1,
        remove_unused_columns=True,
    ),
)

/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_num_proc. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:297: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/h

In [24]:
# debug
if True:
    batch = next(iter(trainer.get_train_dataloader()))
    whole_text = tokenizer.decode(batch["input_ids"][0])
    print(f"{whole_text=}")

    y_label = batch["labels"][0]
    label_text = tokenizer.decode(y_label[y_label!=-100])
    print(f"{label_text=}")

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

whole_text="<|im_start|>system\nYou are a mathematics teacher. Your job is to infer and identify the misconceptions behind the incorrect answers to the questions.<|im_end|>\n<|im_start|>user\n# OBJECTIVE #\nBased on the problem, please select which of the following misconceptions might have led to the student's incorrect answer.\n\nCategories: Number,Decimals,Multiplying and Dividing with Decimals,Divide decimals by 10\nQuestion: \\( 43.2 \\div 10= \\)\nCorrect Answer: \\( 4.32 \\)\nStudent's Answer: \\( 33.2 \\)\nStep-by-Step Solution: The misunderstanding behind the wrong answer \\(33.2\\) likely stems from a confusion about how the decimal point moves when dividing by 10. Here is a step-by-step explanation of the mistake:\n\n1. **Initial Number**: The students start with the number \\(43.2\\).\n2. **Dividing by 10**: When dividing by 10, the decimal point should move one place to the left. However, the students may have mistakenly thought they needed to subtract 10 from the number o

In [25]:
# Start training
trainer_stats = trainer.train()

  0%|          | 0/1367 [00:00<?, ?it/s]The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  0%|          | 5/1367 [00:49<3:41:19,  9.75s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_au

{'loss': 0.8518, 'grad_norm': 0.8664639592170715, 'learning_rate': 0.00029669603524229074, 'epoch': 0.01}


/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  2%|▏         | 25/1367 [04:08<3:34:44,  9.60s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  2%|▏         | 30/1367 [04:59<3:44:20, 10.07s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cp

{'loss': 0.5365, 'grad_norm': 1.1492925882339478, 'learning_rate': 0.0002922907488986784, 'epoch': 0.03}


/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  3%|▎         | 45/1367 [07:30<3:41:20, 10.05s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  4%|▎         | 50/1367 [08:18<3:31:49,  9.65s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cp

...

{'loss': 0.4749, 'grad_norm': 0.6286694407463074, 'learning_rate': 0.00027466960352422903, 'epoch': 0.09}


/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  9%|▉         | 125/1367 [20:49<3:28:24, 10.07s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
 10%|▉         | 130/1367 [21:38<3:23:51,  9.89s/it]/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.

{'loss': 0.4766, 'grad_norm': 0.890263020992279, 'learning_rate': 0.0002702643171806167, 'epoch': 0.1}


/home/yoku/compe/eedi_winners/Eedi_kaggle/src/4.reranking/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [26]:
del model, trainer

gc.collect()
torch.cuda.empty_cache()

## Merge the LoRA model with the base model

In [27]:
lora_path = glob(f"./{output_dir}/checkpoint-*")[0]
lora_path

'./qwne25_72B_fold0/checkpoint-140'

In [28]:
# Load LoRA
model = AutoPeftModelForCausalLM.from_pretrained(lora_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(lora_path)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.96it/s]


In [29]:
model = model.eval()

In [30]:
# Merge QLoRA and the base model
merged = model.merge_and_unload()

In [31]:
# Save merged model
out_path = f"./{output_dir}/merged_model"
merged.save_pretrained(out_path)
tokenizer.save_pretrained(out_path)

('./qwne25_72B_fold0/merged_model/tokenizer_config.json',
 './qwne25_72B_fold0/merged_model/special_tokens_map.json',
 './qwne25_72B_fold0/merged_model/vocab.json',
 './qwne25_72B_fold0/merged_model/merges.txt',
 './qwne25_72B_fold0/merged_model/added_tokens.json',
 './qwne25_72B_fold0/merged_model/tokenizer.json')